<font size="5">Raw Version</font>

Setting up probes:

In [45]:
# desired DNA strand
MBP_seq = "AAAGACAGGCCCTCAGAGTCCGACGAGCTTCTCAGAGTCCGACGAGCTTCAGACCATCCAAGAAGATCCCACAGCAGCTTCCGAAGGCCTGGATGTGATGGCATCACAGAAGAGACCCTCACAGCGACACGGGTCCAAGTACTTGGCCACAGCAAGTACCATGGACCATGCCCGGCATGGCTTCCTCCCAAGGCACAGAGACACGGGCATCCTTGACTCCATCGGGCGCTTCTTTAGCGGTGACAGGGGTGCGCCCAAGCGGGGCTCTGGCAAGGTACCCTGGCTAAAGCAGAGCCGGAGCCCTCTGCCTTCTCATGCCCGCAGCCGTCCCGGGCTGTGCCACATGTACAAGGACTCACACACAAGAACTACCCACTACGGCTCCCTGCCCCAGAAGTCGCAGAGGACCCAAGATGAAAACCCAGTAGTCCACTTCTTCAAGAACATTGTGACACCTCGTACACCCCCTCCATCCCAAGGAAAGGGGAGAGGCCTGTCCCTCAGCAGATTTAGCTGGGGGGCCGAGGGGCAGAAGCCAGGATTTGGCTACGGAGGCAGAGCTTCCGACTATAAATCGGCTCACAAGGGATTCAAGGGGGCCTACGACGCCCAGGGCACGCTTTCCAAAATCTTTAAGCTGGGAGGAAGAGACAGCCGCTCTGGATCTCCCATGGCAAGACGCTGAGAGCCTCCCTGCTCAGCCTTCCCGAATCCTGCCCTCGGCTTCTTAATATAACTGCCTTAAACGTTTAATTCTACTTGCACCAAATAGCTAGTTAGAGCAGACCCTCTCTTAATCCCGTGGGGCTGTGAACGCGGCGGGGCCAGGCCCACGGCACCCTGACTGGCTAAAACTGTTTGTCCCTTTTTATTTGAAGATTGAGTTTCCTCGGGGTCTTCTCTGCCCCGACTTGCTCCCCGTGTACCTTGGTCGACTCCGGAGGTTCAGGTGCACGGACACCCTTTCAAGTTCACCCCTACTCCATCCTCAGACTTTCTTTTCACGGCGAGGCGCACCCCTCCAGCTTCCGTGGGCACTGCGGATAGACAGGCACACCGCCAAGGAGCCAGAGAGCATGGCGCAGGGGACTGTGTGGTCCAGGCTTCCTTTGTTTTCTTTCCCCTAAAGAGCTTTGTTTTTCCTAACAGGATCAGACAGTCTTGGAGTGGCTTACACAACGGGGGCTTGTGGTATGTGAGCACAGGCTGGGCAGCTGTGAGAGTCCAGAGTGGGGTGGCCCTGGGGACGCTTCCAGGCCAGCGGTTCCCTGCACCCCACCAGCTGATTTCGAGCGTGGCAGAGGGAAGGAAAGGGGCGAGCGGGCTGGGCAATGGACCCGACAGGAAACGGGGACTTAGGGGAACACGCTGGAGATGCCATGTGTGGCTGCCGAAGGTCACCATCTCTCCTCAGTGGCTCCCCAGAGCAGGTGCTTTTAAGAACCCTGTTTCCTCTCAGAGCCCAGGGAGAGTCCAAGGACATGGCGCATCAGGAAGTGGGACTGCAGGAGTTCTCTGGTGGCCTCGTGCTGTCCCTCTGGCCACTTCTCACTTTAGGGTGGTCAGCGGCAGCTCGCCATGGCAGTGCCCATTGGTGCACACTAACCTCAGTGGAAAAGTAACCATTCCCTGCCTCTTAGAAAGAACTCATTCTTAGTTTTAGGAGGGTTCCTGTCGCTGAATCAAGTCGCTGCCCTGGATGCAGGGCTGGCCTGGGCGACCCTCCAGGGATGAGGAGCTCAGAATTCCAGTCTTCTAATGTCCACGGACACCTCCCCATCCCTCTAACGTACTGACTATGTCTTTTGATTTAGCATGTCTTCTATAGACCTTCCAAAGAGACCCACACTGGCACTGTCACCCCCTAGGAGGGAAGGTGATGGTTGATGTAGCCCGACGCGCATCTTGTTAATCCGTTCTAATTCCGAGGAGAGTGTGGGTTTAAGATAACACCTATTAATGCATTGCCACAATAATGTGGGGGTAAGAGAAACGCAGGGACGAAACTTCCAGAAACAAACCCTCCAGATCGTTCCACAGGAGTGTTCGCCCTCCGGTGTGACTGAACGACCGACCTTGCCCATGGCTTCATCCAGACAGCACAGCTGCAGTATGGCTGGACAGAAGCACCTACTGTTCTTGGATATTGAAATAAAATAATAAACTTGCAATGATCTTTG"

In [46]:
# complement and reverse DNA or RNA strand
comp = MBP_seq.lower()[::-1].replace("g", "C").replace("c", "G").replace("t", "A").replace("a", "T").replace("u", "A")


In [47]:
# splitting probes
split_probes = []
for n in range(20, 31): 
    for i in range(0, len(comp), n):
        probe = comp[i:i + n]
        
        
        if len(probe) > 20:
            split_probes.append(probe)

# Not sure if we should include file below

# f = open("split_probes.txt", "w")
# f.write(str(split_probes))
# f.close()

PNAS rules:

In [48]:
# PNAS 1
def pnas_filter_1(probe_sequence):
    percent_a = probe_sequence.count('A')/len(probe_sequence)
    return 0.28 < percent_a

In [49]:
# PNAS 2
def pnas_filter_2(probe_sequence):
    A_stack = 'AAAA'
    if A_stack in probe_sequence:
        return False 
    else:
        return True

In [50]:
# PNAS 3
def pnas_filter_3(probe_sequence):
    percent_c = probe_sequence.count('C')/len(probe_sequence)
    return 0.22 < percent_c and 0.28 > percent_c

In [51]:
# PNAS 4
def pnas_filter_4(probe_sequence):
    C_stack = 'CCCC'
    sequence_subset = probe_sequence[0:12]
    if C_stack in sequence_subset:
        return False
    else:
        return True  

In [52]:
# PNAS 5
def pnas_filter_5(probe_sequence):
    sequence_subset = probe_sequence[0:12]
    for p in range(6):
        if sequence_subset[p:p+6].count('C') >= 4:
            return False
    return True

deltaG:

In [53]:
def deltaG1(probe_sequence):
    import pandas as pd
    
    table = {"AA":0.2, "AC":-1.4, "AG":-0.4, "AT":-0.4,
        "CA":-1.6, "CC":-2.3, "CG":-1.4, "CT":-1.3,
        "GA":-1.4, "GC":-2.0, "GG":-1.7, "GT":-1.5, 
        "TA":-0.5, "TC":-1.5, "TG":-1.2, "TT":-0.7}
    
    for p in range(len(probe_sequence)):
        list_of_pairs = [probe_sequence[p:2 + p] for p in range(0, len(probe_sequence))]
        list_of_pairs.pop()
        if len(list_of_pairs) > 0:
            C = (pd.Series(list_of_pairs)).map(table)
            D = list(C)
            return sum(D)

Percent of GC:

In [54]:
def GC_percent(probe_sequence):
    G = probe_sequence.count('G')
    C = probe_sequence.count('C')
    G += G + C
    percent_GC = G/len(probe_sequence)
    return percent_GC

Start and end indexes:

In [55]:
def start_index(probe_sequence):
    if probe in comp:
        start = comp.find(probe_sequence)
        return start

In [56]:
def end_index(probe_sequence):
    if probe in comp:
        end = comp.find(probe_sequence)
        end += len(probe_sequence) - 1
        return end

In [57]:
import pandas as pd

df = pd.DataFrame()
df['probe_sequence'] = split_probes
df['probe_length'] = df['probe_sequence'].apply(len)
df['start'] = df['probe_sequence'].apply(start_index)
df['end'] = df['probe_sequence'].apply(end_index)
df['pnas1'] = df['probe_sequence'].apply(pnas_filter_1)
df['pnas2'] = df['probe_sequence'].apply(pnas_filter_2)
df['pnas3'] = df['probe_sequence'].apply(pnas_filter_3)
df['pnas4'] = df['probe_sequence'].apply(pnas_filter_4)
df['pnas5'] = df['probe_sequence'].apply(pnas_filter_5)
df['deltaG'] = df['probe_sequence'].apply(deltaG1)
df['GC'] = df['probe_sequence'].apply(GC_percent)
df

,probe_sequence,probe_length,start,end,pnas1,pnas2,pnas3,pnas4,pnas5,deltaG,GC
0,CAAAGATCATTGCAAGTTTAT,21,0,20,True,True,False,True,True,-16.4,0.428571
1,TATTTTATTTCAATATCCAAG,21,21,41,True,True,False,True,True,-15.1,0.238095
2,AACAGTAGGTGCTTCTGTCCA,21,42,62,False,True,True,True,True,-24.9,0.714286
3,GCCATACTGCAGCTGTGCTGT,21,63,83,False,True,False,True,True,-27.9,0.857143
4,CTGGATGAAGCCATGGGCAAG,21,84,104,True,True,False,True,True,-23.5,0.952381
...,...,...,...,...,...,...,...,...,...,...,...
857,CATGGTCCATGGTACTTGCTGTGGCCAAGT,30,2010,2039,False,True,True,True,True,-38.2,0.833333
858,ACTTGGACCCGTGTCGCTGTGAGGGTCTCT,30,2040,2069,False,True,True,True,True,-41.7,0.933333
859,TCTGTGATGCCATCACATCCAGGCCTTCGG,30,2070,2099,False,True,False,True,True,-40.9,0.800000
860,AAGCTGCTGTGGGATCTTCTTGGATGGTCT,30,2100,2129,False,True,False,True,True,-36.0,0.833333


Overlap:

In [62]:
def overlapping_intervals(a, b):
    if a[0] < b[0]:
        first = a
        second = b
    elif a[0] == b[0]:
        return True
    else:
        first = b
        second = a
        
    return first[1] > second[0]

In [63]:
def check_probe_fits(probe_candidate, probe_list):
    for probe in probe_list:
        if overlapping_intervals([probe['start'], probe['end']], [probe_candidate['start'], probe_candidate['end']]):
            return False
    return True

def find_nonoverlapping_probes_around_median(df):
    
    # Calculate median delta G
    median = df['deltaG'].median()
    
    # Order rows in probes dataframe by distance to median with Pandas-function
    for probe in range(len(df['deltaG'])):
        result_index = df['deltaG'].sub(median).abs().argsort()[0:].tolist()
        ordered_rows = df.iloc[result_index]

    # Iteratively build list of non-overlapping probes around the median
    final_probe_list = [df.iloc[result_index[0]]]
    for index in result_index[1:]:
        probe_candidate = df.iloc[index]
        if check_probe_fits(probe_candidate, final_probe_list):
            final_probe_list.append(probe_candidate)
    
    return pd.DataFrame(final_probe_list)

In [64]:
def check_probe_fits(probe_candidate, probe_list):
    for probe in probe_list:
        if overlapping_intervals([probe['start'], probe['end']], [probe_candidate['start'], probe_candidate['end']]):
            return False
    return True

def find_nonoverlapping_probes_around_median(df):
    
    # Calculate median delta G
    median = df['deltaG'].median()
    
    # Order rows in probes dataframe by distance to median with Pandas-function
    for probe in range(len(df['deltaG'])):
        result_index = df['deltaG'].sub(median).abs().argsort()[0:].tolist()
        ordered_rows = df.iloc[result_index]

    # Iteratively build list of non-overlapping probes around the median
    final_probe_list = [df.iloc[result_index[0]]]
    for index in result_index[1:]:
        probe_candidate = df.iloc[index]
        if check_probe_fits(probe_candidate, final_probe_list):
            final_probe_list.append(probe_candidate)
    
    return pd.DataFrame(final_probe_list)

<font size="5">Input Version</font>

In [61]:
# would it be input or sys.arg???

# raw_seq = 'CAAAGATCATTGCAAGTTTATTATTTTATTTCAATATCCAAGAACAGTAGGTGCTTCTGTCCAGCCATACTGCAGCTGTGCTGTCTGGATGAAGCCATGGGCAAGGTCGGTCGTTCAGTCACACCGGAGGGCGAACACTCCTGTGGAACGATCTGGAGGGTTTGTTTCTGGAAGTTTCGTCCCTGCGTTTCTCTTACCCCCACATTATTGTGGCAATGCATTAATAGGTGTTATCTTAAACCCACACTCTCCTCGGAATTAGAA'
# comp = raw_seq.lower()[::-1].replace("g", "C").replace("c", "G").replace("t", "A").replace("a", "T").replace("u", "A")

# if len(comp) < 600:
#     print('Target RNA too short. Minimum length is ___ bases')
#     print('Your target RNA is ' + str(len(raw_seq)) + ' bases long.')
# else:
#     pass